# Variable misuse detection

VarMisuse at node level

In [3]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [4]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from torch import nn
from datetime import datetime

Using backend: pytorch


# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

There are a lot of parameters. Ones that might be of interest are marked with `***`.

In [34]:
config = get_config(
    # tokenizer
    tokenizer_path="sentencepiece_bpe.model", # *** path to sentencepiece model
    
    # dataset parameters
    data_path="large_graph",             # *** path to node type
    use_node_types=False,                # node types currently not supported
    use_edge_types=True,                 # whether to use edge types
    filter_edges=None,                   # None or list of edge type names
    self_loops=False,                    # whether to use self loops
    train_frac=0.8,                      # *** fraction of nodes to use for training
    random_seed=42,                      # random seed for splitting dataset int o train test validation
    min_count_for_objectives=5,          # *** minimum frequency of targets
    no_global_edges=False,               # remove global edges
    remove_reverse=False,                # remove reverse edges
    custom_reverse=None,                 # None or list of edges, for which reverse edges should be created (use together with `remove_reverse`)
    
    # training parameters
    model_output_dir="large_graph",      # *** directory to save checkpoints and training data
    batch_size=128,                     # *** 
    sampling_neighbourhood_size=10,      # number of dependencies to sample for each node
    neg_sampling_factor=1,               # *** number of negative samples for each positive sample
    epochs=10,                           # *** number of epochs
    elem_emb_size=100,                   # *** dimensionality of target embeddings (for node name prediction)
    pretraining_phase=0,                 # number of epochs for pretraining
    embedding_table_size=200000,         # *** embedding table size for subwords
    save_checkpoints=False,              # set to False if checkpoints are not needed
    save_each_epoch=False,               # save each epoch, useful in case of studying model behavior
    measure_scores=True,                 # *** measure ranking scores during evaluation
    dilate_scores=200,                   # downsampling factor for measuring scores to make evaluation faster
    objectives="node_clf",               # type of objective
    force_w2v_ns=True,                   # negative sampling strategy
    gpu=-1,                              # gpuid
    restore_state=False,
    pretrained=None,
    
    # model parameters
    node_emb_size=100,                   # *** dimensionality of node embeddings
    h_dim=100,                           # *** should match to node dimensionality
    n_layers=5,
    num_bases=10,                        # number of bases for computing parmetwer weights for different edge types
    dropout=0.2,                         # *** 
    use_self_loop=True,                  #
    activation="tanh",                   # *** 
    learning_rate=1e-3,                  # *** 
    use_gcn_checkpoint=True,
    use_att_checkpoint=True,
    use_gru_checkpoint=True
)

In [35]:
save_config(config, "config.yaml")
config = load_config("config.yaml")
config

{'DATASET': {'custom_reverse': None,
  'data_path': 'large_graph',
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'no_global_edges': False,
  'random_seed': 42,
  'remove_reverse': False,
  'restricted_id_pool': None,
  'self_loops': False,
  'subgraph_id_column': 'mentioned_in',
  'subgraph_partition': None,
  'train_frac': 0.8,
  'use_edge_types': True,
  'use_node_types': False},
 'MODEL': {'activation': 'tanh',
  'dropout': 0.2,
  'h_dim': 100,
  'n_layers': 5,
  'node_emb_size': 100,
  'num_bases': 10,
  'use_att_checkpoint': True,
  'use_gcn_checkpoint': True,
  'use_gru_checkpoint': True,
  'use_self_loop': True},
 'TOKENIZER': {'tokenizer_path': 'sentencepiece_bpe.model'},
 'TRAINING': {'batch_size': 128,
  'dilate_scores': 200,
  'early_stopping': False,
  'early_stopping_tolerance': 20,
  'elem_emb_size': 100,
  'embedding_table_size': 200000,
  'epochs': 10,
  'external_dataset': None,
  'force_w2v_ns': True,
  'gpu': -1,
  'learning_rate': 0.001,
  'measure_scor

# Create Dataset

In [36]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]},
)

# Declare target loading function (labels)

In [37]:
unpersist("misuse_labels.json")

,src,dst
0,659505,correct
1,505406,misused
2,476754,correct
3,209536,correct
4,723591,correct
...,...,...
96128,284057,correct
96129,157734,correct
96130,594115,misused
96131,393944,misused


In [38]:
def load_var_misuse():
    from SourceCodeTools.code.data.dataset.reader import load_data
    
    nodes = dataset.nodes
    
    var_misuse_labels = unpersist("misuse_labels.json").query("src in @node_ids", local_dict={"node_ids": nodes["id"]})
    
    return var_misuse_labels

In [39]:
load_var_misuse()

,src,dst
5,103078,correct
20,107642,correct
21,835267,misused
25,418107,correct
28,125246,correct
...,...,...
96103,993273,correct
96106,158471,correct
96116,139737,correct
96124,127758,misused


In [40]:
dataset.nodes

,id,type,name,mentioned_in,string,type_backup,embeddable,embeddable_name,typed_id,train_mask,val_mask,test_mask,global_graph_id
208998,3,node_,__future__,<NA>,<NA>,module,True,__future__,0,True,False,False,0
212857,4,node_,__future__.absolute_import,<NA>,<NA>,global_variable,True,__future__.absolute_import,1,True,False,False,1
89611,5,node_,__future__.division,<NA>,<NA>,global_variable,True,__future__.division,2,True,False,False,2
156128,6,node_,__future__.print_function,<NA>,<NA>,global_variable,True,__future__.print_function,3,True,False,False,3
263839,7,node_,sys,<NA>,<NA>,module,True,sys,4,False,True,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,2499247,node_,Call_0x16d22e24634faccd,2499242,self.assertFalse(is_finalizing()),Call,True,Call,324213,False,False,True,324213
259178,2499248,node_,is_finalizing@FunctionDef_0x16d22e246314e4ad,2499242,<NA>,mention,True,is_finalizing,324214,True,False,False,324214
131932,2499249,node_,Call_0x16d22e2463277fc6,2499242,is_finalizing(),Call,True,Call,324215,True,False,False,324215
146867,2499250,node_,test_basic@ClassDef_0x16d22e24634a3399,2499243,<NA>,mention,True,test_basic,324216,True,False,False,324216


# Define objectives

Currenlty objectives for node classification (`NodeClassifierObjective`), and name-based node embedding training `SubwordEmbedderObjective`.

![]("examples/figures/img1.png)


One or several objectives could be used

In [41]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        self.objectives.append(
            NodeClassifierObjective(
                "VarMisuse",
                self.graph_model, self.node_embedder, dataset.nodes,
                load_var_misuse,                                                   # need to define this function
                self.device, self.sampling_neighbourhood_size, self.batch_size,
                tokenizer_path=tokenizer_path, target_emb_size=self.elem_emb_size, 
                masker=None,                                                            # masker is not needed here
                measure_scores=False,
                dilate_scores=self.trainer_params["dilate_scores"]
            )
        )

In [23]:
%tensorboard --logdir "large_graph"

Reusing TensorBoard on port 6006 (pid 13488), started 0:03:54 ago. (Use '!kill 13488' to kill it.)

In [ ]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    tokenizer_path=config["TOKENIZER"]["tokenizer_path"],
    trainer=Trainer
)

Epoch 0:   0%|                                                                                                                  | 0/101 [00:00<?, ?it/s]

Number of nodes 324218


Epoch 1:   0%|                                                                                                                  | 0/101 [00:00<?, ?it/s]

Epoch: 0, Time: 1178 s
{'Accuracy/test/VarMisuse_': 0.7548338210702341,
 'Accuracy/train/VarMisuse_': 0.9411764705882353,
 'Accuracy/train_avg/VarMisuse': 0.7607109056493885,
 'Accuracy/val/VarMisuse_': 0.7554086538461539,
 'Loss/test/VarMisuse_': 0.561052354482504,
 'Loss/train/VarMisuse_': 0.3619568347930908,
 'Loss/train_avg/VarMisuse': 0.5551836056874531,
 'Loss/val/VarMisuse_': 0.56044418995197}


Epoch 2:   0%|                                                                                                                  | 0/101 [00:00<?, ?it/s]

...

Epoch 3:   0%|                                                                                                                  | 0/101 [00:00<?, ?it/s]

Epoch: 2, Time: 1320 s
{'Accuracy/test/VarMisuse_': 0.7548338210702341,
 'Accuracy/train/VarMisuse_': 0.9411764705882353,
 'Accuracy/train_avg/VarMisuse': 0.7617164749563191,
 'Accuracy/val/VarMisuse_': 0.7554086538461539,
 'Loss/test/VarMisuse_': 0.5628140156085675,
 'Loss/train/VarMisuse_': 0.340114027261734,
 'Loss/train_avg/VarMisuse': 0.550746511114706,
 'Loss/val/VarMisuse_': 0.5612786641487708}


Epoch 4:   0%|                                                                                                                  | 0/101 [00:00<?, ?it/s]

Epoch: 3, Time: 1288 s
{'Accuracy/test/VarMisuse_': 0.7548338210702341,
 'Accuracy/train/VarMisuse_': 0.9411764705882353,
 'Accuracy/train_avg/VarMisuse': 0.7617164749563191,
 'Accuracy/val/VarMisuse_': 0.7554086538461539,
 'Loss/test/VarMisuse_': 0.5628632788474743,
 'Loss/train/VarMisuse_': 0.3328274190425873,
 'Loss/train_avg/VarMisuse': 0.5502336270148211,
 'Loss/val/VarMisuse_': 0.5621340068487021}


Epoch 4:  31%|████████████████████████████████▏                                                                        | 31/101 [06:49<14:25, 12.37s/it]